In [1]:
import os
import zipfile
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-03-08 23:45:04.677268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../../../lib:
2023-03-08 23:45:04.677291: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# !pip install kaggle

In [13]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv(data_path)

# Extract the label column and convert it to a NumPy array
labels = data['label'].to_numpy()

# Extract the image data columns and convert them to a NumPy array
images = data.iloc[:, 1:].to_numpy()

# Normalize the image data to values between 0 and 1
images = images / 255.0

# Reshape the image data to 28x28x1 arrays (for compatibility with the model)
images = images.reshape(-1, 28, 28, 1)

# Convert the labels to one-hot encoded categorical variables
labels = to_categorical(labels, num_classes=26)

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10)

# Save the model
model.save('sign_language_model.h5')


Epoch 1/10


2023-03-08 23:40:45.115760: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 68879104 exceeds 10% of free system memory.


 15/687 [..............................] - ETA: 7s - loss: 3.2434 - accuracy: 0.0333

2023-03-08 23:40:45.554077: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2023-03-08 23:40:45.554165: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2023-03-08 23:40:45.570851: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.


687/687 [==============================] - 7s 10ms/step - loss: 1.4267 - accuracy: 0.5626 - val_loss: 0.5280 - val_accuracy: 0.8330
Epoch 2/10
687/687 [==============================] - 7s 10ms/step - loss: 0.2771 - accuracy: 0.9216 - val_loss: 0.1521 - val_accuracy: 0.9610
Epoch 3/10
687/687 [==============================] - 7s 10ms/step - loss: 0.0846 - accuracy: 0.9811 - val_loss: 0.0553 - val_accuracy: 0.9905
Epoch 4/10
687/687 [==============================] - 7s 10ms/step - loss: 0.0240 - accuracy: 0.9975 - val_loss: 0.0153 - val_accuracy: 0.9982
Epoch 5/10
687/687 [==============================] - 7s 10ms/step - loss: 0.0101 - accuracy: 0.9995 - val_loss: 0.0064 - val_accuracy: 0.9995
Epoch 6/10
687/687 [==============================] - 7s 10ms/step - loss: 0.0234 - accuracy: 0.9934 - val_loss: 0.0443 - val_accuracy: 0.9856
Epoch 7/10
687/687 [==============================] - 7s 10ms/step - loss: 0.0055 - accuracy: 0.9992 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 8/10

In [2]:
# Load the trained model
model = tf.keras.models.load_model('sign_language_model.h5')

# Define the classes
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

# Define the region of interest
roi = (100, 100, 300, 300)

# Initialize the video camera
cap = cv2.VideoCapture(0)

# Loop over each frame
while True:
    # Read the frame
    ret, frame = cap.read()

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Draw the region of interest
    cv2.rectangle(frame, roi[:2], roi[2:], (0, 255, 0), 2)

    # Extract the region of interest
    x1, y1, x2, y2 = roi
    roi_frame = frame[y1:y2, x1:x2]

    # Preprocess the image
    roi_frame = cv2.resize(roi_frame, (28, 28))
    roi_frame = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
    roi_frame = np.array(roi_frame, dtype=np.float32)
    roi_frame = roi_frame / 255.0
    roi_frame = roi_frame.reshape(1, 28, 28, 1)

    # Predict the sign language gesture
    prediction = model.predict(roi_frame)

    # Get the class name
    class_name = classes[np.argmax(prediction)]

    # Display the predicted class name
    cv2.putText(frame, class_name, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Sign Language Recognition', frame)

    # Wait for the user to press 'q' to exit
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video camera
cap.release()

# Close all windows
cv2.destroyAllWindows()


2023-03-08 23:45:15.984551: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../../../lib:
2023-03-08 23:45:15.984579: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-08 23:45:15.984595: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kirito): /proc/driver/nvidia/version does not exist
2023-03-08 23:45:15.984844: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step
